# Catboost wieght и учет факторов

Вместо удаления дорогих квартир, попробуйте использовать взвешивание данных при обучении модели CatBoost.  Присвойте большим весам данные из элитного сегмента.  Это позволит модели уделять больше внимания этим данным и лучше их "запоминать".  CatBoost позволяет задавать веса наблюдений через параметр sample_weight

In [ ]:
import numpy as np
from catboost import CatBoostRegressor

#  Создаем веса.  Например,  увеличиваем вес для элитных квартир в 5 раз
weights = np.where(df['статус жк'] == 'элитный', 5, 1)

model = CatBoostRegressor(iterations=1000,  # другие параметры
                         loss_function='RMSE',
                         verbose=100,
                         cat_features=categorical_features)

model.fit(X_train, y_train, sample_weight=weights)

1. Пропорциональное взвешивание:
Этот подход нацелен на балансировку классов, придавая больший вес менее представленным классам.  Вычисляем обратную частоту каждого класса:

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# Подсчет количества объектов в каждом классе
class_counts = df['статус ЖК'].value_counts()

# Вычисление весов, обратных частотам
weights = 1.0 / class_counts[df['статус ЖК']]

# Применение весов
model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features)
model.fit(X_train, y_train, sample_weight=weights.values)

2.  Взвешивание на основе вашей экспертной оценки:
Если у вас есть экспертная информация о важности каждого класса для модели,  вы можете задать веса вручную:

In [ ]:
# Задаем веса вручную, например:
weights_dict = {'нет': 1, 'престижный': 2, 'престижный+': 5}  # 'престижный+' имеет наибольший вес

weights = df['статус ЖК'].map(weights_dict)

model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features)
model.fit(X_train, y_train, sample_weight=weights.values)

3.  Комбинированный подход:

In [ ]:
class_counts = df['статус ЖК'].value_counts()
weights = 1.0 / class_counts[df['статус ЖК']]

expert_weights = {'нет': 1, 'престижный': 1.2, 'престижный+': 1.5} # коэффициенты экспертной оценки
weights = weights * df['статус ЖК'].map(expert_weights)

model.fit(X_train, y_train, sample_weight=weights.values)

CatBoost использует эти веса для каждого наблюдения в обучающей выборке, но только в контексте частоты встречаемости категорий в столбце "статус ЖК".  Для других признаков веса не применяются.

1. Взвешивание по каждому признаку отдельно, затем усреднение:

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# Взвешивание для "статус ЖК"
class_counts_status = df['статус ЖК'].value_counts()
weights_status = 1.0 / class_counts_status[df['статус ЖК']]

# Взвешивание для "до мкад/за мкад"
class_counts_mkad = df['до мкад/за мкад'].value_counts()
weights_mkad = 1.0 / class_counts_mkad[df['до мкад/за мкад']]

# Усреднение весов (можно использовать другие методы агрегации, например, геометрическое среднее)
weights = (weights_status + weights_mkad) / 2

model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features)
model.fit(X_train, y_train, sample_weight=weights.values)

2.  Взвешивание по комбинациям признаков:

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# Создаем новый столбец с комбинацией категорий
df['комбинация'] = df['статус ЖК'] + '_' + df['до мкад/за мкад']

# Взвешивание по комбинациям
class_counts_combo = df['комбинация'].value_counts()
weights_combo = 1.0 / class_counts_combo[df['комбинация']]

model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features)
model.fit(X_train, y_train, sample_weight=weights_combo.values)

Этот метод создаёт новый признак, представляющий собой комбинацию значений из "статус ЖК" и "до мкад/за мкад", и затем вычисляет веса на основе частоты встречаемости этих комбинаций.  Этот подход лучше, чем простое усреднение, потому что он учитывает взаимосвязь между признаками.

3.  Использование экспертных знаний:

In [ ]:
weight_table = {
    ('нет', 'внутри мкад'): 1,
    ('нет', 'за мкад'): 0.8,
    ('престижный', 'внутри мкад'): 3,
    ('престижный', 'за мкад'): 2,
    ('престижный+', 'внутри мкад'): 5,
    ('престижный+', 'за мкад'): 4,
    # ... другие комбинации ...
}
weights = df.apply(lambda row: weight_table[(row['статус ЖК'], row['до мкад/за мкад'])], axis=1)

1. Валидация и веса:

Вы правы, при обучении модели CatBoost в метод fit веса добавляются только для обучающей выборки (X_train, y_train).  Валидационная выборка (которую CatBoost использует внутренне для ранней остановки и предотвращения переобучения) не использует эти веса.  CatBoost сам управляет внутренней валидацией; вы не передаете веса для валидационной выборки в fit.


2. Хранение весов в модели:

Обученная модель CatBoost  не хранит  веса, используемые во время обучения. Веса используются только в процессе обучения для корректировки функции потерь. После обучения модель представляет собой набор параметров (например, веса деревьев в градиентном бустинге), которые оптимально аппроксимируют зависимость между признаками и целевой переменной, учитывая данные с весами.


3. Удаление признака "статус ЖК":

Вы правильно заметили, что в примере я убрал признак "статус ЖК" из X  (X = df.drop(['цена', 'статус ЖК'], axis=1)).  Это сделано потому, что веса, которые мы вычисляем, основаны на распределении категорий именно в этом признаке.  Если оставить "статус ЖК" в X, то модель будет использовать его как обычный признак, и это может привести к дублированию информации и проблемам с обучением.  Модель будет пытаться учитывать вклад "статус ЖК" дважды: как обычный категориальный признак и неявно, через веса, что исказит результаты.


4. Расчет метрик и веса:

Вы правы, при расчете метрик на тестовой выборке веса не используются.  Метрики должны отражать обобщающую способность модели на новых, невиденных данных, независимо от весов, использованных при обучении.


5. Как модель "знает" веса (после drop):

Модель  не "знает"  веса после обучения.  Веса применяются только во время процесса обучения для изменения функции потерь и, следовательно, влияния на подбор оптимальных параметров модели.  После обучения веса не используются.  Прогнозы на тестовых и новых данных делаются с помощью обученной модели без учёта весов, используемых при обучении.


Исправленный код (без удаления "статус ЖК"):

В действительности, удаление 'статус ЖК' не всегда необходимо.  Если вы хотите, чтобы модель учитывала 'статус ЖК' как обычный предиктор, то лучше оставить его. В этом случае вы будете использовать веса для балансировки классов, а не для изменения влияния признака "статус ЖК".


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ... (загрузка данных и вычисление весов - как в предыдущем примере) ...

# Разделение признаков (X) и целевой переменной (y) -  'статус ЖК' ОСТАЕТСЯ
X = df.drop('цена', axis=1)
y = df['цена']


X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, test_size=0.2, random_state=42
)

categorical_features_indices = [X.columns.get_loc(col) for col in ['район', 'округ', 'город/область', 'до мкад/за мкад', 'статус ЖК']]


model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features_indices)
model.fit(X_train, y_train, sample_weight=weights_train)

# ... (прогнозирование и расчет метрик - как в предыдущем примере) ...

Существует несколько способов учесть информацию о двух "сверх-элитных" ЖК в вашей модели CatBoost, чтобы улучшить прогнозирование цен для них. Ключевая идея — добавить информацию о дополнительной категории элитности или создать новые признаки, которые бы отражали особенности этих ЖК.

Метод 1: Добавление новой категории в фактор "статус ЖК"

Самый простой и понятный способ — добавить новую категорию в фактор "статус ЖК", например, "престиж++". Это позволит модели напрямую выучить специфику ценообразования для этой новой категории.

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor

# ... (Загрузка данных и создание модели - как в предыдущих примерах) ...

# Идентификация сверх-элитных ЖК (замените на ваши условия)
super_elite_zhk = ['ЖК1', 'ЖК2']  # Названия или ID сверх-элитных ЖК

# Добавление новой категории
df['статус ЖК'] = df['статус ЖК'].apply(lambda x: 'престиж++' if df['Название ЖК'].isin(super_elite_zhk) else x)


# Обновление списка категориальных признаков (очень важно!)
categorical_features_indices = [df.columns.get_loc(col) for col in categorical_columns]

# Обучение модели (с обновленными категориями)
model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features_indices)
model.fit(X_train, y_train, sample_weight = weights_train) # weights_train - Ваши веса, если используются
#... (прогнозирование и оценка)

Метод 2: Создание новых признаков

Вместо добавления категории, можно создать новые бинарные признаки, которые указывают на принадлежность к этим двум ЖК.

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
# ... (Загрузка данных и создание модели - как в предыдущих примерах) ...

# Создание новых признаков
df['ЖК1'] = df['Название ЖК'].apply(lambda x: 1 if x == 'ЖК1' else 0)
df['ЖК2'] = df['Название ЖК'].apply(lambda x: 1 if x == 'ЖК2' else 0)

# Обучение модели (с новыми признаками)
model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100) # cat_features может не понадобиться, если новые признаки числовые
model.fit(X_train, y_train, sample_weight = weights_train) # weights_train - Ваши веса, если используются

#... (прогнозирование и оценка)

Метод 3: Использование взвешивания (более сложный вариант):

Можно попробовать использовать взвешивание, но не просто для классов "статус ЖК", а для отдельных объектов. Если вы уверены, что данные по этим двум ЖК особенно важны, можно присвоить им значительно больший вес в обучающей выборке.

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# ... (Загрузка данных и создание модели - как в предыдущих примерах) ...

# Взвешивание данных
weights = np.ones(len(df))
weights[df['Название ЖК'].isin(super_elite_zhk)] = 10  # Увеличиваем вес для сверх-элитных ЖК в 10 раз

model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100)
model.fit(X_train, y_train, sample_weight=weights)

# ... (прогнозирование и оценка)

Метод 4: Комбинация:

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# ... (Загрузка данных и создание модели - как в предыдущих примерах) ...

# Идентификация сверх-элитных ЖК
super_elite_zhk = ['ЖК1', 'ЖК2']

# Добавление новой категории
df['статус ЖК'] = df['статус ЖК'].apply(lambda x: 'престиж++' if df['Название ЖК'].isin(super_elite_zhk) else x)

# Обновление списка категориальных признаков
categorical_features_indices = [df.columns.get_loc(col) for col in categorical_columns]

# Задание весов
weights_dict = {'нет': 1, 'престижный': 2, 'престижный+': 3, 'престиж++': 5}
weights = df['статус ЖК'].map(weights_dict)

# Обучение модели
model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features_indices)
model.fit(X_train, y_train, sample_weight=weights)

# ... (прогнозирование и оценка) ...


Важные замечания:

• Выбор весов:  Веса 1, 2, 3, 5 — это пример.  Вам, возможно, потребуется экспериментировать с разными значениями весов, чтобы найти оптимальное соотношение.  Начните с небольших различий в весах и постепенно увеличивайте, следя за качеством прогноза.
• Оценка качества:  Тщательно отслеживайте качество модели на тестовой выборке после внесения изменений.  Убедитесь, что комбинированный подход (добавление категории + взвешивание) действительно улучшает качество прогнозирования, особенно для сверх-элитных ЖК.  Не исключено, что простого добавления категории "престиж++" будет достаточно.
• Переобучение:  Чрезмерное взвешивание может привести к переобучению.  Обращайте внимание на метрики качества на тестовой выборке.
• Feature Importance:  После обучения модели, посмотрите на feature_importances_.  Это поможет понять, насколько эффективно модель использует новый признак и веса.

Реализация расчета веса через средние значения удельной цены:

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# ... (Загрузка данных, добавление категории "престиж++", как в предыдущем примере) ...

# Расчет удельной цены за кв.м для каждой категории
df['цена_кв_м'] = df['цена'] / df['площадь']

# Группировка данных по "статус ЖК" и расчет средней удельной цены
avg_prices = df.groupby('статус ЖК')['цена_кв_м'].mean()

# Расчет доли каждой категории в общей сумме цен
total_price = avg_prices.sum()
weights = avg_prices / total_price

# Применение весов к данным
weights_series = df['статус ЖК'].map(weights)

# Обучение модели CatBoost
model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features_indices)
model.fit(X_train, y_train, sample_weight=weights_series)

# ... (прогнозирование и оценка) ...


Минусы среднего:

• Чувствительность к выбросам:  Если в какой-либо категории есть несколько очень дорогих квартир (выбросы), это может сильно исказить среднюю удельную цену и, следовательно, вес этой категории.  Это может привести к переобучению модели на выбросах.
• Зависимость от распределения:  Веса сильно зависят от распределения цен в каждой категории.  Если распределение цен сильно асимметричное, то средняя цена может быть не очень представительной.
• Неучет других факторов:  Этот метод учитывает только удельную цену, игнорируя другие важные факторы, которые могут влиять на цену квартиры (например, расположение, год постройки, состояние).

Плюсы использования медианы:

• Устойчивость к выбросам: Главное преимущество медианы – её устойчивость к выбросам. В отличие от среднего, медиана не сильно меняется при наличии экстремальных значений. В данных о ценах на недвижимость выбросы (очень дорогие или очень дешевые квартиры) встречаются часто, и использование медианы помогает сгладить их влияние на расчет весов. Это делает модель более устойчивой к искажениям, вызванным аномальными значениями.
• Более представительная оценка: В случае асимметричных распределений цен медиана часто является более представительной мерой центральной тенденции, чем среднее. Среднее может быть сильно смещено в сторону выбросов, тогда как медиана остается ближе к типичным значениям.
• Проще интерпретировать: Медиана проще интерпретируется, чем среднее, особенно для неспециалистов. Медиана — это значение, которое делит выборку пополам.


Минусы использования медианы:

• Меньшая чувствительность к изменениям: Медиана менее чувствительна к изменениям в данных, чем среднее. Если есть небольшие, но систематические изменения в ценах, среднее будет их отражать лучше, чем медиана.
• Более сложные вычисления: В некоторых случаях расчет медианы может быть немного сложнее, чем среднего, хотя в Python это не составляет проблемы.
• Потеря информации: Использование медианы означает, что мы игнорируем часть информации о распределении цен. Среднее учитывает все значения, тогда как медиана фокусируется только на центральной части распределения.

Взвешивание на основе квантилей:

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# ... (Загрузка данных, добавление категории "престиж++") ...

def quantile_weights(df, column, n_quantiles=10):
    """Вычисляет веса на основе квантилей."""
    quantiles = pd.qcut(df[column], q=n_quantiles, labels=False, duplicates='drop')
    weights = (n_quantiles - quantiles) / n_quantiles #Квантили ближе к максимуму получают больший вес
    return weights

# Расчет весов на основе квантилей цены
weights = quantile_weights(df, 'цена')

# Обучение модели CatBoost
model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features_indices)
model.fit(X_train, y_train, sample_weight=weights)

# ... (прогнозирование и оценка) ...

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# ... (Загрузка данных, добавление категории "престиж++") ...

def quantile_weights_by_category(df, column, category_column, n_quantiles=10):
    """Вычисляет веса на основе квантилей для каждой категории."""
    weights = []
    for category in df[category_column].unique():
        df_subset = df[df[category_column] == category]
        quantiles = pd.qcut(df_subset[column], q=n_quantiles, labels=False, duplicates='drop')
        category_weights = (n_quantiles - quantiles) / n_quantiles  #Высшие квантили получают больший вес
        weights.extend(category_weights.tolist())
    return np.array(weights)


# Расчет весов на основе квантилей цены для каждой категории "статус ЖК"
weights = quantile_weights_by_category(df, 'цена', 'статус ЖК', n_quantiles=10)

# Обучение модели CatBoost
model = CatBoostRegressor(iterations=1000, loss_function='RMSE', verbose=100, cat_features=categorical_features_indices)
model.fit(X_train, y_train, sample_weight=weights)

# ... (прогнозирование и оценка) ...


In [ ]:
import numpy as np
from catboost import CatBoostRegressor

# 1. Определение функции для назначения весов на основе ценового диапазона
def get_weight(price):
    if price <= 3000000:  # До 3 млн
        return 0.5  # Понижаем вес, так как переоцениваются
    elif price >= 20000000:  # От 20 млн
        return 2.0  # Увеличиваем вес, так как недооцениваются
    else:
        return 1.0  # Оставляем без изменений

# 2. Создание списка весов для каждой записи в обучающих данных
train_weights = [get_weight(price) for price in y_train]

# 3. Обучение модели CatBoostRegressor с использованием sample_weight
model = CatBoostRegressor(
    iterations=100,
    loss_function='RMSE',  # или другая подходящая функция потерь для регрессии
    eval_metric='RMSE',  # или другая метрика для оценки
    verbose=False, # Отключаем вывод информации во время обучения
    random_seed=42
)

model.fit(X_train, y_train, verbose=False, sample_weight=train_weights)

# 4. Оценка модели на отложенной выборке (validation)
y_pred = model.predict(X_val)

# Рассчет метрик на разных диапазонах цен для анализа смещения.
# (Пример)
def calculate_metrics_by_price_range(y_true, y_pred, price_ranges):
    metrics = {}
    for lower_bound, upper_bound in price_ranges:
        # Находим индексы квартир, попадающих в этот диапазон
        indices = np.where((y_true >= lower_bound) & (y_true < upper_bound))[0]

        if len(indices) > 0:
            # Выбираем только те значения, которые попадают в диапазон
            y_true_range = y_true[indices]
            y_pred_range = y_pred[indices]

            # Рассчитываем метрики (например, MAE, RMSE)
            mae = np.mean(np.abs(y_true_range - y_pred_range))
            rmse = np.sqrt(np.mean((y_true_range - y_pred_range)**2))

            metrics[f"{lower_bound}-{upper_bound}"] = {"MAE": mae, "RMSE": rmse}
        else:
            metrics[f"{lower_bound}-{upper_bound}"] = "No data in range"

    return metrics

price_ranges = [
    (0, 3000000),
    (3000000, 6000000),
    (6000000, 9000000),
    (9000000, 12000000),
    (12000000, 15000000),
    (15000000, 20000000),
    (20000000, float('inf'))  # Бесконечность для цен выше 20 млн
]

metrics_by_range = calculate_metrics_by_price_range(y_val, y_pred, price_ranges)
print("Метрики по диапазонам цен:\n", metrics_by_range)

# 5. Анализ результатов и итеративное изменение весов
# ...

1. get_weight(price): Функция, которая принимает цену квартиры и возвращает соответствующий вес. Внутри функции определяется логика назначения весов в зависимости от ценового диапазона. Этот код присваивает вес 0.5 квартирам до 3 млн, вес 2.0 квартирам от 20 млн и вес 1.0 всем остальным. Вы можете настроить эту функцию в соответствии с вашими потребностями.

2. train_weights: Создается список весов для каждой квартиры в обучающем наборе данных. Этот список создается путем итерации по y_train (который содержит фактические цены квартир) и применения функции get_weight() к каждой цене.

3. CatBoostRegressor(sample_weight=train_weights): При обучении модели CatBoostRegressor мы передаем список train_weights в аргумент sample_weight функции fit(). Это указывает CatBoost использовать эти веса при обучении.

•   Вес 1 (1.0): Вес 1 означает, что данному образцу (строке данных) уделяется *стандартное*, или *обычное*, внимание при обучении модели. Это как если бы вы не использовали sample_weight вообще. Образцы с весом 1 рассматриваются как "типичные" или "представительные" данные.  Их вклад в функцию потерь будет соответствовать их ошибке, без какого-либо увеличения или уменьшения.

Пояснение в контексте ваших данных о квартирах:

•   Если вы присваиваете вес 1 квартирам в ценовом диапазоне "от 6 до 9 миллионов рублей", это означает, что вы не хотите специально "толкать" модель к тому, чтобы она лучше или хуже предсказывала цены этих квартир, по сравнению с тем, как она это делала бы без использования sample_weight.  Вы считаете, что модель уже достаточно хорошо работает с этим ценовым диапазоном, или что у вас нет оснований уделять этим квартирам больше или меньше внимания.

Аналогия:

Представьте, что вы учитель, и у вас есть группа учеников.

•   Вес > 1 (например, 2): Это как если бы вы решили уделить *больше* внимания конкретному ученику, потому что он испытывает трудности с определенной темой, или потому что он очень талантлив и вы хотите помочь ему раскрыть свой потенциал.
•   Вес < 1 (например, 0.5): Это как если бы вы решили уделить *меньше* внимания ученику, потому что он уже хорошо справляется с материалом, или потому что он отвлекает других учеников в классе.
•   Вес 1: Это как если бы вы относились к ученику как к *обычному* ученику, уделяя ему стандартное количество внимания.

Ключевой момент:

Важно помнить, что веса — это *относительные* значения. Значение имеет не абсолютное значение веса (например, 2), а его *отношение* к другим весам в вашем наборе данных.  Если бы вы умножили все веса на 2, то это, вероятно, не оказало бы никакого эффекта на обучение модели (если только это не привело бы к численным проблемам).  Важно, чтобы разница между весами отражала вашу уверенность в том, насколько важно для модели правильно предсказывать разные типы образцов.